<a href="https://colab.research.google.com/github/dochev/CryptoTrade/blob/master/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !git init && git remote add origin https://github.com/dochev/CryptoTrade.git && git pull origin master
!git remote set-url origin https://github.com/dochev/CryptoTrade.git
# !git clone https://github.com/dochev/CryptoTrade.git
# !pip install -r requirements.txt

In [2]:
!git fetch origin master

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 5 (delta 3), reused 5 (delta 3), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/dochev/CryptoTrade
 * branch            master     -> FETCH_HEAD
   3b50b9b..86c790d  master     -> origin/master


In [3]:
!git pull origin master

From https://github.com/dochev/CryptoTrade
 * branch            master     -> FETCH_HEAD
fatal: refusing to merge unrelated histories


In [4]:
%ls

cli.py             LICENSE                  run-with-docker*
config/            optimize.py              sample_data/
CryptoTrade/       README.md                tensortrade/
data/              requirements.base.txt    test/
dev-with-docker*   requirements.no-gpu.txt  Vagrantfile
docker/            requirements.tests.txt   visualization.gif
Experiments.ipynb  requirements.txt
lib/               run-tests-with-docker*


In [5]:
%cd CryptoTrade

/content/CryptoTrade


In [6]:
%ls

CONTRIBUTING.md  LICENSE      requirements.txt  Welcome_To_Colaboratory.ipynb
docker/          Makefile     setup.cfg
docs/            MANIFEST.in  setup.py
examples/        README.md    tensortrade/


In [0]:
# !pip install -r requirements.txt

In [0]:
import os
import sys
import warnings

def warn(*args, **kwargs):
    pass

warnings.warn = warn
warnings.simplefilter(action='ignore', category=FutureWarning)

sys.path.append(os.path.dirname(os.path.abspath('')))

from tensortrade.environments import TradingEnvironment
from tensortrade.exchanges.simulated import FBMExchange
from tensortrade.actions import DiscreteActionStrategy
from tensortrade.rewards import SimpleProfitStrategy

exchange = FBMExchange()
action_strategy = DiscreteActionStrategy()
reward_strategy = SimpleProfitStrategy()

env = TradingEnvironment(exchange=exchange,
                         action_strategy=action_strategy,
                         reward_strategy=reward_strategy)

obs = env.reset()
sell_price = 1e9
stop_price = -1

# print('Initial portfolio: ', exchange.portfolio)

# for i in range(1000):
#     action = 0 if obs['close'] < sell_price else 18
#     action = 19 if obs['close'] < stop_price else action
    
#     if i == 0 or portfolio['BTC'] == 0:
#         action = 16
#         sell_price = obs['close'] + (obs['close'] / 50)
#         stop_price = obs['close'] - (obs['close'] / 50)
    
#     obs, reward, done, info = env.step(action)
#     executed_trade = info['executed_trade']
#     filled_trade = info['filled_trade']
#     portfolio = exchange.portfolio
    
#     print('Obs: ', obs)
#     print('Reward: ', reward)
#     print('Portfolio: ', portfolio)
#     print('Trade executed: ', executed_trade.trade_type, executed_trade.price, executed_trade.amount)
#     print('Trade filled: ', filled_trade.trade_type, filled_trade.price, filled_trade.amount)

In [0]:
# !pip install tensorforce

In [12]:
import sys
import os
import warnings

def warn(*args, **kwargs):
    pass

warnings.warn = warn
warnings.simplefilter(action='ignore', category=FutureWarning)

import gym
import numpy as np

from tensorforce.agents import Agent, DQNAgent
from tensorforce.execution import Runner
from tensorforce.environments import OpenAIGym

sys.path.append(os.path.dirname(os.path.abspath('')))

from tensortrade.environments import TradingEnvironment
from tensortrade.exchanges.simulated import FBMExchange
from tensortrade.actions import DiscreteActionStrategy
from tensortrade.rewards import SimpleProfitStrategy

exchange = FBMExchange(times_to_generate=100000)
action_strategy = DiscreteActionStrategy()
reward_strategy = SimpleProfitStrategy()

env = TradingEnvironment(exchange=exchange,
                         action_strategy=action_strategy,
                         reward_strategy=reward_strategy,
                         feature_pipeline=None)

# env = OpenAIGym('CartPole-v0')

agent_config = {
    "type": "dqn_agent",

    "update_mode": {
        "unit": "timesteps",
        "batch_size": 64,
        "frequency": 4
    },
    
    "memory": {
        "type": "replay",
        "capacity": 10000,
        "include_next_states": True
    },

    "optimizer": {
        "type": "clipped_step",
        "clipping_value": 0.1,
        "optimizer": {
            "type": "adam",
            "learning_rate": 1e-3
        }
    },

    "discount": 0.999,
    "entropy_regularization": None,
    "double_q_model": True,

    "target_sync_frequency": 1000,
    "target_update_weight": 1.0,

    "actions_exploration": {
        "type": "epsilon_anneal",
        "initial_epsilon": 0.5,
        "final_epsilon": 0.,
        "timesteps": 1000000000
    },

    "saver": {
        "directory": None,
        "seconds": 600
    },
    "summarizer": {
        "directory": None,
        "labels": ["graph", "total-loss"]
    },
    "execution": {
        "type": "single",
        "session_config": None,
        "distributed_spec": None
    }
}

network_spec = [
    dict(type='dense', size=64, activation='tanh'),
    dict(type='dense', size=32, activation='tanh')
]
  
agent = DQNAgent(
        states=env.states,
        actions=env.actions,
        network=network_spec,
        batch_size=64
#         update=dict(unit='timesteps', batch_size=64),
#         objective='policy_gradient',
#         reward_estimation=dict(horizon=20)
    )

# Initialize the agent
agent.initialize()

# Create the runner
runner = Runner(agent=agent, environment=env)

# Start learning
max_episodes = 300
runner.run(max_episodes)
runner.close()

# Print statistics
print("Learning finished. Total episodes: {ep}. Average reward of last 100 episodes: {ar}.".format(
    ep=runner.episode,
    ar=np.mean(runner.episode_rewards))
)

AssertionError: ignored

In [0]:
import gym
import pandas as pd
import numpy as np
from gym import spaces
from sklearn import preprocessing

class BitcoinTradingEnv(gym.Env):
  """A Bitcoin trading environment for OpenAI gym"""
  metadata = {'render.modes': ['live', 'file', 'none']}
  scaler = preprocessing.MinMaxScaler()
  viewer = None
  def __init__(self, df, lookback_window_size=50, 
                         commission=0.00075,  
                         initial_balance=10000,
                         serial=False):
    super(BitcoinTradingEnv, self).__init__()
    self.df = df.dropna().reset_index()
    self.lookback_window_size = lookback_window_size
    self.initial_balance = initial_balance
    self.commission = commission
    self.serial = serial
  # Actions of the format Buy 1/10, Sell 3/10, Hold, etc.
    self.action_space = spaces.MultiDiscrete([3, 10])
  # Observes the OHCLV values, net worth, and trade history
    self.observation_space = spaces.Box(low=0, high=1, shape=(10, 
                    lookback_window_size + 1), dtype=np.float16)
    
  def reset(self):
    self.balance = self.initial_balance
    self.net_worth = self.initial_balance
    self.btc_held = 0
    self._reset_session()
    self.account_history = np.repeat([
      [self.net_worth],
      [0],
      [0],
      [0],
      [0]
    ], self.lookback_window_size + 1, axis=1)
    self.trades = []
    return self._next_observation()

  MAX_TRADING_SESSION = 100000  # ~2 months

  def _reset_session(self):
    self.current_step = 0
    if self.serial:
      self.steps_left = len(self.df) - self.lookback_window_size - 1
      self.frame_start = self.lookback_window_size
    else:
      self.steps_left = np.random.randint(1, MAX_TRADING_SESSION)
      self.frame_start = np.random.randint(
           self.lookback_window_size, len(self.df) - self.steps_left)
    self.active_df = self.df[self.frame_start -   
         self.lookback_window_size:self.frame_start + self.steps_left]

  def _next_observation(self):
    end = self.current_step + self.lookback_window_size + 1
    obs = np.array([
      self.active_df['Open'].values[self.current_step:end],  
      self.active_df['High'].values[self.current_step:end],
      self.active_df['Low'].values[self.current_step:end],
      self.active_df['Close'].values[self.current_step:end],
      self.active_df['Volume_(BTC)'].values[self.current_step:end],
    ])
    scaled_history = self.scaler.fit_transform(self.account_history)
    obs = np.append(obs, scaled_history[:, -(self.lookback_window_size
                                                       + 1):], axis=0)
    return obs

  def step(self, action):
    current_price = self._get_current_price() + 0.01
    self._take_action(action, current_price)
    self.steps_left -= 1
    self.current_step += 1
    if self.steps_left == 0:
      self.balance += self.btc_held * current_price
      self.btc_held = 0
      self._reset_session()
    obs = self._next_observation()
    reward = self.net_worth
    done = self.net_worth <= 0
    return obs, reward, done, {}

  def _take_action(self, action, current_price):
    action_type = action[0]
    amount = action[1] / 10
    btc_bought = 0
    btc_sold = 0
    cost = 0
    sales = 0
    if action_type < 1:
      btc_bought = self.balance / current_price * amount
      cost = btc_bought * current_price * (1 + self.commission)
      self.btc_held += btc_bought
      self.balance -= cost
    elif action_type < 2:
      btc_sold = self.btc_held * amount
      sales = btc_sold * current_price  * (1 - self.commission)
      self.btc_held -= btc_sold
      self.balance += sales
    if btc_sold > 0 or btc_bought > 0:
      self.trades.append({
        'step': self.frame_start+self.current_step,
        'amount': btc_sold if btc_sold > 0 else btc_bought,
        'total': sales if btc_sold > 0 else cost,
        'type': "sell" if btc_sold > 0 else "buy"
      })
    self.net_worth = self.balance + self.btc_held * current_price
    self.account_history = np.append(self.account_history, [
      [self.net_worth],
      [btc_bought],
      [cost],
      [btc_sold],
      [sales]
    ], axis=1)


#   from datetime import datetime
#   date_labels = np.array([datetime.utcfromtimestamp(x).strftime(
#   '%Y-%m-%d %H:%M') for x in self.df['Timestamp'].values[step_range]])

  def render(self, mode='human', **kwargs):
    if mode == 'human':
      if self.viewer == None:
        self.viewer = BitcoinTradingGraph(self.df,
                                          kwargs.get('title', None))
      self.viewer.render(self.frame_start + self.current_step,
                         self.net_worth,
                         self.trades,
                         window_size=self.lookback_window_size)

In [20]:
df = pd.read_csv("../data/coinbaseUSD_1-min_data_2014-12-01_to_2018-11-11.csv")
slice_point = int(len(df) - 100000)
train_df = df[:slice_point]
test_df = df[slice_point:]

train_env = DummyVecEnv([lambda: BitcoinTradingEnv(train_df, 
                         commission=0, serial=False)])
test_env = DummyVecEnv([lambda: BitcoinTradingEnv(test_df, 
                        commission=0, serial=True)])

NameError: ignored

In [21]:
%ls

CONTRIBUTING.md  LICENSE      requirements.txt  Welcome_To_Colaboratory.ipynb
docker/          Makefile     setup.cfg
docs/            MANIFEST.in  setup.py
examples/        README.md    tensortrade/


In [0]:
model = PPO2(MlpPolicy,
             train_env,
             verbose=1, 
             tensorboard_log="./tensorboard/")
model.learn(total_timesteps=50000)